Посмотрим на ассоциативные правила

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
from subprocess import check_output
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import tqdm, tqdm_notebook

In [4]:
def process_split(x):
    set_, count = x.strip().split(' #SUP: ')
    set_ = set_.strip()
    set_ = ' '.join([str(x) for x in sorted([int(x) for x in set_.split(' ')])])
    return set_, len(set_.split(' ')), int(count)

def load_file(fname):
    with open(fname, 'r') as f:
        loaded = f.readlines()
    return [process_split(x) for x in loaded if process_split(x)[1] == 2]

# spmf processed filename
associative_rules_path = '../data/associative_rules.txt'

In [57]:
products = pd.read_csv('../data/input/products.csv')
products = products.rename(columns={'product_id':'item_id', 'product_name':'item_name'})[['item_id', 'item_name']]

orders = pd.read_csv('../data/input/order_products__prior.csv', usecols = ['order_id', 'product_id'])

ovc = orders['product_id'].value_counts()

In [68]:
rules = load_file(associative_rules_path)
rules = pd.DataFrame(rules)
rules.columns = ['item_id_A', 'item_id_B', 'support']

In [69]:
rules['lift'] = [s / (ovc[a] * ovc[b]) for a, b, s in zip(rules['item_id_A'], rules['item_id_B'], rules['support'])]

rules = rules.merge(products, how = 'left', left_on = 'item_id_A', right_on='item_id').rename(columns = {'item_name': 'product_A'}).drop(columns = ['item_id_A', 'item_id'])\
             .merge(products, how = 'left', left_on = 'item_id_B', right_on='item_id').rename(columns = {'item_name': 'product_B'}).drop(columns = ['item_id_B', 'item_id'])
rules = rules.sort_values('lift', ascending=False)[['product_A', 'product_B', 'support', 'lift']]

In [70]:
rules.head(20)

,product_A,product_B,support,lift
0,Organic Strawberry Chia Lowfat 2% Cottage Cheese,Organic Cottage Cheese Blueberry Acai Chia,306,9.447744
1,Grain Free Chicken Formula Cat Food,Grain Free Turkey Formula Cat Food,318,5.970560
2,Organic Fruit Yogurt Smoothie Mixed Berry,Apple Blueberry Fruit Yogurt Smoothie,349,5.569327
3,Nonfat Strawberry With Fruit On The Bottom Gre...,"0% Greek, Blueberry on the Bottom Yogurt",409,5.339895
4,Organic Grapefruit Ginger Sparkling Yerba Mate,Cranberry Pomegranate Sparkling Yerba Mate,351,5.266226
5,Baby Food Pouch - Roasted Carrot Spinach & Beans,"Baby Food Pouch - Butternut Squash, Carrot & C...",332,5.162524
6,Unsweetened Whole Milk Mixed Berry Greek Yogurt,Unsweetened Whole Milk Blueberry Greek Yogurt,438,5.034039
7,Uncured Cracked Pepper Beef,Chipotle Beef & Pork Realstick,410,4.913031
8,Organic Mango Yogurt,Organic Whole Milk Washington Black Cherry Yogurt,334,4.337266
10,Raspberry Essence Water,Unsweetened Pomegranate Essence Water,366,4.185361
